# Packages

In [20]:
# Use autoreload to automatically reload modules
%load_ext autoreload
%autoreload 2

In [21]:
# Setup paths
import rootutils
import xarray as xr
import pandas as pd
root_path = rootutils.find_root()

In [22]:
def img_interactive(da):
    import ipywidgets as widgets
    import matplotlib.pyplot as plt

    def plot_time_steps(time_index1, time_index2):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
        
        # Left image
        da.isel(time=time_index1).plot.imshow(robust=True, ax=ax1)
        ax1.set_title(f"{str(da.time.values[time_index1])[:10]}")
        ax1.axis('off')
        
        # Right image  
        da.isel(time=time_index2).plot.imshow(robust=True, ax=ax2)
        ax2.set_title(f"{str(da.time.values[time_index2])[:10]}")
        ax2.axis('off')
        
        plt.tight_layout()
        plt.show()

    widgets.interact(
        plot_time_steps, 
        time_index1=widgets.IntSlider(min=0, max=len(da.time)-1, step=1, value=0, description='Left Image'),
        time_index2=widgets.IntSlider(min=0, max=len(da.time)-1, step=1, value=1, description='Right Image')
    )

In [23]:
deforest_df = pd.read_parquet(root_path / "data" / "processed" / "radd" / "south_america_combined_clean.parquet")
deforest_df

,geometry,component_id,size_pixels,area_hectares,earliest_alert,latest_alert,duration_days,centroid_x,centroid_y,confidence,tile_name,date_raster_file
0,b'\x01\x01\x00\x00\x00\x03\x16k;l\xbeH\xc0QXZ\...,32,109,1.342989,2021-08-29,2021-09-22,24,-49.487678,-8.150868,high,00N_050W_radd_alerts,sample_07_dates.tif
1,b'\x01\x01\x00\x00\x00zw\xf0\xf05\xcdH\xc0\r\x...,111,312,3.844152,2025-05-16,2025-06-09,24,-49.603209,-8.475450,high,00N_050W_radd_alerts,sample_07_dates.tif
2,b'\x01\x01\x00\x00\x00\x0cVy\x99\xed\xc6H\xc0~...,128,104,1.281384,2025-06-21,2025-07-15,24,-49.554126,-8.481419,high,00N_050W_radd_alerts,sample_07_dates.tif
3,b'\x01\x01\x00\x00\x00\xec^\xe8\xdacYF\xc0\xca...,25,1220,15.031620,2020-05-27,2020-06-25,29,-44.698360,-5.696953,high,00N_050W_radd_alerts,sample_11_dates.tif
4,b'\x01\x01\x00\x00\x00\x91\xee\x8f@\x06kF\xc0\...,145,106,1.306026,2022-11-06,2022-11-18,12,-44.836128,-5.717239,high,00N_050W_radd_alerts,sample_11_dates.tif
...,...,...,...,...,...,...,...,...,...,...,...,...
2672,b'\x01\x01\x00\x00\x00\xbe\xb1\x8de\xe7\xaaO\x...,242,105,1.293705,2020-10-10,2020-11-09,30,-63.335187,-20.671847,high,20S_070W_radd_alerts,sample_37_dates.tif
2673,b'\x01\x01\x00\x00\x00\xb01#\xdf\x0e\xacO\xc0\...,265,182,2.242422,2020-10-10,2020-11-09,30,-63.344204,-20.675970,high,20S_070W_radd_alerts,sample_37_dates.tif
2674,b'\x01\x01\x00\x00\x00\xaa\x7f\xe2I-\xacO\xc0\...,286,162,1.996002,2020-10-10,2020-11-09,30,-63.345132,-20.680279,high,20S_070W_radd_alerts,sample_37_dates.tif
2675,b'\x01\x01\x00\x00\x00R\xd5n\x8fM\xadO\xc0PC;\...,372,146,1.798866,2020-10-04,2020-10-28,24,-63.353929,-20.704417,high,20S_070W_radd_alerts,sample_37_dates.tif


In [24]:

i = 33
print(f"Earliest alert: {deforest_df['earliest_alert'].iloc[i]}")
print(f"Latest alert: {deforest_df['latest_alert'].iloc[i]}")
da_s2 = xr.open_dataarray(root_path / "data" / "processed" / "sen_data" / f"event_{i}" /"da_s2.nc" ) 
da_s2_rgb = da_s2.sel(band=["B04", "B03", "B02"])
img_interactive(da_s2_rgb)

Earliest alert: 2024-11-17
Latest alert: 2024-11-29


FileNotFoundError: [Errno 2] No such file or directory: '/net/home/lmaecker/own_projects/burned_embedder/data/processed/sen_data/event_33/da_s2.nc'